In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")

application_df_original = application_df
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at SUCCESS count to determine if stratifying is needed
type_count = application_df.IS_SUCCESSFUL.value_counts()
type_count

1    18261
0    16038
Name: IS_SUCCESSFUL, dtype: int64

In [5]:
# Look at APPLICATION TYPE value counts for binning
type_count = application_df.APPLICATION_TYPE.value_counts()
type_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Determine which values to replace if counts are less than ..?
replace_type = list(type_count[type_count<1000].index)

# Replace in dataframe
for t in replace_type:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(t,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df.CLASSIFICATION.value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_count[classification_count<1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
income_count = application_df.INCOME_AMT.value_counts()
income_count

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [10]:
# Convert income amount to integer based on low number in range

income_amt_int = []

for i, row in application_df.iterrows():
  if application_df['INCOME_AMT'][i] == '0':
    income_amt_int.append(0)
  elif application_df['INCOME_AMT'][i] == '1-9999':
    income_amt_int.append(1)
  elif application_df['INCOME_AMT'][i] == '10000-24999':
    income_amt_int.append(10000)
  elif application_df['INCOME_AMT'][i] == '25000-99999':
    income_amt_int.append(25000)
  elif application_df['INCOME_AMT'][i] == '100000-499999':
    income_amt_int.append(100000)
  elif application_df['INCOME_AMT'][i] == '1M-5M':
    income_amt_int.append(1000000)
  elif application_df['INCOME_AMT'][i] == '5M-10M':
    income_amt_int.append(5000000)
  elif application_df['INCOME_AMT'][i] == '10M-50M':
    income_amt_int.append(10000000)
  elif application_df['INCOME_AMT'][i] == '50M+':
    income_amt_int.append(50000000)
  else:
    income_amt_int.append(0)

application_df['INCOME_AMT'] = income_amt_int

pre_drop_df = application_df
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,Other,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000,N,142590,1


In [11]:

import os
import time
from tensorflow.keras.callbacks import ModelCheckpoint

In [12]:
# Plan is to drop one column at a time.
# creating list of columns to drop
columns = application_df.columns[:-1]
# creating dataframe to hold results
results_df = pd.DataFrame(columns=['dropped_column', 'loss', 'accuracy'])

In [13]:
# running model on whole dataframe (no rows beyond EIN and Name dropped)

application_df_new2 = application_df

# Generate our categorical variable lists
cat_var = application_df_new2.dtypes[application_df_new2.dtypes == 'object'].index.tolist()

# Encoding Data and merging into df
enc = OneHotEncoder(sparse=False)

encode_df = pd.DataFrame(enc.fit_transform(application_df_new2[cat_var]))

encode_df.columns = enc.get_feature_names_out(cat_var)

application_df_new = application_df_new2.merge(encode_df, left_index=True, right_index=True).drop(columns=cat_var)

# Splitting data
y = application_df_new['IS_SUCCESSFUL']
X = application_df_new.drop(columns=['IS_SUCCESSFUL'])

X_train, X_test, y_train, y_test = train_test_split(X, y)

# Creating scaler
scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Define the model 
number_input_features = len(X.columns)
hidden_nodes_layer1 = 24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Define the checkpoint path and filenames
os.makedirs("checkpoints_opt2/NONE_dropped/",exist_ok=True)
checkpoint_path = "checkpoints_opt2/NONE_dropped/weights.{epoch:02d}.hdf5"

# create callback to saves weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq= int(len(X_train)/32*5))

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50,callbacks=[cp_callback])

model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

results_df.loc[len(results_df.index)] = ['None', model_loss, model_accuracy]

Epoch 1/50


/Users/mattzabriskie/miniconda3/envs/mlenv_forge/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
2023-03-27 23:42:47.737083: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


804/804 [==============================] - 1s 369us/step - loss: 0.5889 - accuracy: 0.7105
Epoch 2/50
804/804 [==============================] - 0s 373us/step - loss: 0.5664 - accuracy: 0.7259
Epoch 3/50
804/804 [==============================] - 0s 346us/step - loss: 0.5635 - accuracy: 0.7261
Epoch 4/50
804/804 [==============================] - 0s 370us/step - loss: 0.5626 - accuracy: 0.7271
Epoch 5/50
801/804 [============================>.] - ETA: 0s - loss: 0.5606 - accuracy: 0.7287
Epoch 5: saving model to checkpoints_opt2/NONE_dropped/weights.05.hdf5
804/804 [==============================] - 0s 323us/step - loss: 0.5610 - accuracy: 0.7283
Epoch 6/50
804/804 [==============================] - 0s 356us/step - loss: 0.5610 - accuracy: 0.7282
Epoch 7/50
804/804 [==============================] - 0s 342us/step - loss: 0.5602 - accuracy: 0.7283
Epoch 8/50
804/804 [==============================] - 0s 352us/step - loss: 0.5592 - accuracy: 0.7293
Epoch 9/50
804/804 [===================

In [14]:
# running the keras nn model on df, dropping one column at a time

for column in columns:

    #creating df with dropped column (pre encoding)
    application_df_new = application_df.drop(columns=column)

    # Generate our categorical variable lists
    cat_var = application_df_new.dtypes[application_df_new.dtypes == 'object'].index.tolist()

    # Encoding and merging into df
    enc = OneHotEncoder(sparse=False)

    encode_df = pd.DataFrame(enc.fit_transform(application_df_new[cat_var]))

    encode_df.columns = enc.get_feature_names_out(cat_var)

    application_df_new = application_df_new.merge(encode_df, left_index=True, right_index=True).drop(columns=cat_var)

    # Splitting data
    y = application_df_new['IS_SUCCESSFUL']
    X = application_df_new.drop(columns=['IS_SUCCESSFUL'])

    X_train, X_test, y_train, y_test = train_test_split(X, y)

    # Scaling Data
    scaler = StandardScaler()

    X_scaler = scaler.fit(X_train)

    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)

    # Define the model
    number_input_features = len(X.columns)
    hidden_nodes_layer1 = 24
    hidden_nodes_layer2 = 12

    nn = tf.keras.models.Sequential()

    # First hidden layer
    nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

    # Second hidden layer
    nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

    # Output layer
    nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

    # Define the checkpoint path and filenames
    os.makedirs(f"checkpoints_opt2/{column}_dropped/",exist_ok=True)
    checkpoint_path = f"checkpoints_opt2/{column}_dropped/weights." + "{epoch:02d}.hdf5"

    # create callback to saves weights every 5 epochs
    cp_callback = ModelCheckpoint(
        filepath=checkpoint_path,
        verbose=1,
        save_weights_only=True,
        save_freq= int(len(X_train)/32*5))
    
    # Train the model
    fit_model = nn.fit(X_train_scaled, y_train, epochs=50,callbacks=[cp_callback])

    model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

    # save results to results df
    results_df.loc[len(results_df.index)] = [column, model_loss, model_accuracy]
results_single = results_df.copy()
results_df.sort_values('accuracy', ascending=False)

Epoch 1/50


/Users/mattzabriskie/miniconda3/envs/mlenv_forge/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


804/804 [==============================] - 1s 367us/step - loss: 0.6061 - accuracy: 0.6864
Epoch 2/50
804/804 [==============================] - 0s 358us/step - loss: 0.5882 - accuracy: 0.6995
Epoch 3/50
804/804 [==============================] - 0s 372us/step - loss: 0.5860 - accuracy: 0.7002
Epoch 4/50
804/804 [==============================] - 0s 352us/step - loss: 0.5846 - accuracy: 0.7020
Epoch 5/50
733/804 [==========================>...] - ETA: 0s - loss: 0.5847 - accuracy: 0.6997
Epoch 5: saving model to checkpoints_opt2/APPLICATION_TYPE_dropped/weights.05.hdf5
804/804 [==============================] - 0s 350us/step - loss: 0.5836 - accuracy: 0.7008
Epoch 6/50
804/804 [==============================] - 0s 356us/step - loss: 0.5829 - accuracy: 0.7030
Epoch 7/50
804/804 [==============================] - 0s 357us/step - loss: 0.5830 - accuracy: 0.7021
Epoch 8/50
804/804 [==============================] - 0s 404us/step - loss: 0.5820 - accuracy: 0.7036
Epoch 9/50
804/804 [=======

/Users/mattzabriskie/miniconda3/envs/mlenv_forge/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


804/804 [==============================] - 1s 418us/step - loss: 0.6326 - accuracy: 0.6392
Epoch 2/50
804/804 [==============================] - 0s 382us/step - loss: 0.6202 - accuracy: 0.6530
Epoch 3/50
804/804 [==============================] - 0s 319us/step - loss: 0.6192 - accuracy: 0.6530
Epoch 4/50
804/804 [==============================] - 0s 353us/step - loss: 0.6182 - accuracy: 0.6552
Epoch 5/50
735/804 [==========================>...] - ETA: 0s - loss: 0.6158 - accuracy: 0.6550
Epoch 5: saving model to checkpoints_opt2/AFFILIATION_dropped/weights.05.hdf5
804/804 [==============================] - 0s 348us/step - loss: 0.6170 - accuracy: 0.6543
Epoch 6/50
804/804 [==============================] - 0s 407us/step - loss: 0.6168 - accuracy: 0.6566
Epoch 7/50
804/804 [==============================] - 0s 356us/step - loss: 0.6162 - accuracy: 0.6560
Epoch 8/50
804/804 [==============================] - 0s 347us/step - loss: 0.6163 - accuracy: 0.6576
Epoch 9/50
804/804 [============

...

/Users/mattzabriskie/miniconda3/envs/mlenv_forge/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


804/804 [==============================] - 1s 368us/step - loss: 0.5926 - accuracy: 0.7037
Epoch 2/50
804/804 [==============================] - 0s 366us/step - loss: 0.5679 - accuracy: 0.7225
Epoch 3/50
804/804 [==============================] - 0s 369us/step - loss: 0.5659 - accuracy: 0.7216
Epoch 4/50
804/804 [==============================] - 0s 368us/step - loss: 0.5638 - accuracy: 0.7232
Epoch 5/50
758/804 [===========================>..] - ETA: 0s - loss: 0.5629 - accuracy: 0.7248
Epoch 5: saving model to checkpoints_opt2/SPECIAL_CONSIDERATIONS_dropped/weights.05.hdf5
804/804 [==============================] - 0s 403us/step - loss: 0.5630 - accuracy: 0.7246
Epoch 6/50
804/804 [==============================] - 0s 330us/step - loss: 0.5620 - accuracy: 0.7255
Epoch 7/50
804/804 [==============================] - 0s 337us/step - loss: 0.5615 - accuracy: 0.7247
Epoch 8/50
804/804 [==============================] - 0s 366us/step - loss: 0.5610 - accuracy: 0.7266
Epoch 9/50
804/804 [=

/Users/mattzabriskie/miniconda3/envs/mlenv_forge/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


804/804 [==============================] - 1s 390us/step - loss: 0.5892 - accuracy: 0.7127
Epoch 2/50
804/804 [==============================] - 0s 384us/step - loss: 0.5670 - accuracy: 0.7252
Epoch 3/50
804/804 [==============================] - 0s 419us/step - loss: 0.5637 - accuracy: 0.7275
Epoch 4/50
804/804 [==============================] - 0s 354us/step - loss: 0.5618 - accuracy: 0.7279
Epoch 5/50
739/804 [==========================>...] - ETA: 0s - loss: 0.5611 - accuracy: 0.7289
Epoch 5: saving model to checkpoints_opt2/ASK_AMT_dropped/weights.05.hdf5
804/804 [==============================] - 0s 347us/step - loss: 0.5612 - accuracy: 0.7284
Epoch 6/50
804/804 [==============================] - 0s 337us/step - loss: 0.5596 - accuracy: 0.7301
Epoch 7/50
804/804 [==============================] - 0s 384us/step - loss: 0.5591 - accuracy: 0.7294
Epoch 8/50
804/804 [==============================] - 0s 365us/step - loss: 0.5583 - accuracy: 0.7295
Epoch 9/50
804/804 [================

,dropped_column,loss,accuracy
8,SPECIAL_CONSIDERATIONS,0.552131,0.739708
6,STATUS,0.552820,0.734344
7,INCOME_AMT,0.554848,0.732128
0,None,0.558676,0.727464
4,USE_CASE,0.559129,0.725598
9,ASK_AMT,0.563264,0.724315
5,ORGANIZATION,0.571947,0.723499
3,CLASSIFICATION,0.579552,0.722099
1,APPLICATION_TYPE,0.582664,0.707872
2,AFFILIATION,0.616628,0.650962


In [15]:
# attempting to drop two columns, using the best result from single drop as starting point
# Dropping the top result from whole

results_sorted = results_single.sort_values('accuracy', ascending=False)
result_list = list(results_sorted['dropped_column'])
first_drop = result_list[0]

application_df = application_df.drop(columns=first_drop)

In [16]:

# defining column list to cycle through
columns = application_df.columns[:-1]

for column in columns:

    application_df_new = application_df.drop(columns=column)

    # Generate our categorical variable lists
    cat_var = application_df_new.dtypes[application_df_new.dtypes == 'object'].index.tolist()

    # Encoding DF and merging
    enc = OneHotEncoder(sparse=False)

    encode_df = pd.DataFrame(enc.fit_transform(application_df_new[cat_var]))

    encode_df.columns = enc.get_feature_names_out(cat_var)

    application_df_new = application_df_new.merge(encode_df, left_index=True, right_index=True).drop(columns=cat_var)

    # Splitting data
    y = application_df_new['IS_SUCCESSFUL']
    X = application_df_new.drop(columns=['IS_SUCCESSFUL'])

    X_train, X_test, y_train, y_test = train_test_split(X, y)

    # Scaling Data
    scaler = StandardScaler()

    X_scaler = scaler.fit(X_train)

    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)

    # Define the model
    number_input_features = len(X.columns)
    hidden_nodes_layer1 = 24
    hidden_nodes_layer2 = 12

    nn = tf.keras.models.Sequential()

    # First hidden layer
    nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

    # Second hidden layer
    nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

    # Output layer
    nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

    # Define the checkpoint path and filenames
    os.makedirs(f"checkpoints_opt2/{first_drop}_and_{column}_dropped/",exist_ok=True)
    checkpoint_path = f"checkpoints_opt2/{first_drop}_and_{column}_dropped/weights." + "{epoch:02d}.hdf5"

    # create callback to saves weights every 5 epochs
    cp_callback = ModelCheckpoint(
        filepath=checkpoint_path,
        verbose=1,
        save_weights_only=True,
        save_freq= int(len(X_train)/32*5))
    
    # Train the model
    fit_model = nn.fit(X_train_scaled, y_train, epochs=50,callbacks=[cp_callback])

    model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

    results_df.loc[len(results_df.index)] = [f"{first_drop}, {column}", model_loss, model_accuracy]

results_df.sort_values('accuracy', ascending=False)

Epoch 1/50


/Users/mattzabriskie/miniconda3/envs/mlenv_forge/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


804/804 [==============================] - 1s 393us/step - loss: 0.6071 - accuracy: 0.6910
Epoch 2/50
804/804 [==============================] - 0s 442us/step - loss: 0.5888 - accuracy: 0.6999
Epoch 3/50
804/804 [==============================] - 0s 407us/step - loss: 0.5851 - accuracy: 0.7034
Epoch 4/50
804/804 [==============================] - 0s 421us/step - loss: 0.5834 - accuracy: 0.7063
Epoch 5/50
770/804 [===========================>..] - ETA: 0s - loss: 0.5824 - accuracy: 0.7052
Epoch 5: saving model to checkpoints_opt2/SPECIAL_CONSIDERATIONS_and_APPLICATION_TYPE_dropped/weights.05.hdf5
804/804 [==============================] - 0s 463us/step - loss: 0.5826 - accuracy: 0.7055
Epoch 6/50
804/804 [==============================] - 0s 476us/step - loss: 0.5820 - accuracy: 0.7068
Epoch 7/50
804/804 [==============================] - 0s 393us/step - loss: 0.5815 - accuracy: 0.7065
Epoch 8/50
804/804 [==============================] - 0s 345us/step - loss: 0.5808 - accuracy: 0.7079


/Users/mattzabriskie/miniconda3/envs/mlenv_forge/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


804/804 [==============================] - 1s 412us/step - loss: 0.6388 - accuracy: 0.6372
Epoch 2/50
804/804 [==============================] - 0s 392us/step - loss: 0.6218 - accuracy: 0.6508
Epoch 3/50
804/804 [==============================] - 0s 481us/step - loss: 0.6186 - accuracy: 0.6540
Epoch 4/50
804/804 [==============================] - 0s 401us/step - loss: 0.6182 - accuracy: 0.6538
Epoch 5/50
688/804 [========================>.....] - ETA: 0s - loss: 0.6176 - accuracy: 0.6530
Epoch 5: saving model to checkpoints_opt2/SPECIAL_CONSIDERATIONS_and_AFFILIATION_dropped/weights.05.hdf5
804/804 [==============================] - 0s 438us/step - loss: 0.6173 - accuracy: 0.6528
Epoch 6/50
804/804 [==============================] - 0s 393us/step - loss: 0.6168 - accuracy: 0.6540
Epoch 7/50
804/804 [==============================] - 0s 394us/step - loss: 0.6166 - accuracy: 0.6544
Epoch 8/50
804/804 [==============================] - 0s 415us/step - loss: 0.6161 - accuracy: 0.6542
Epoch

...

/Users/mattzabriskie/miniconda3/envs/mlenv_forge/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


804/804 [==============================] - 1s 407us/step - loss: 0.5932 - accuracy: 0.7014
Epoch 2/50
804/804 [==============================] - 0s 449us/step - loss: 0.5697 - accuracy: 0.7215
Epoch 3/50
804/804 [==============================] - 0s 401us/step - loss: 0.5665 - accuracy: 0.7240
Epoch 4/50
804/804 [==============================] - 0s 425us/step - loss: 0.5649 - accuracy: 0.7234
Epoch 5/50
757/804 [===========================>..] - ETA: 0s - loss: 0.5638 - accuracy: 0.7238
Epoch 5: saving model to checkpoints_opt2/SPECIAL_CONSIDERATIONS_and_INCOME_AMT_dropped/weights.05.hdf5
804/804 [==============================] - 0s 405us/step - loss: 0.5644 - accuracy: 0.7242
Epoch 6/50
804/804 [==============================] - 0s 445us/step - loss: 0.5635 - accuracy: 0.7237
Epoch 7/50
804/804 [==============================] - 0s 402us/step - loss: 0.5629 - accuracy: 0.7240
Epoch 8/50
804/804 [==============================] - 0s 401us/step - loss: 0.5627 - accuracy: 0.7243
Epoch 

/Users/mattzabriskie/miniconda3/envs/mlenv_forge/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


804/804 [==============================] - 1s 324us/step - loss: 0.5917 - accuracy: 0.7046
Epoch 2/50
804/804 [==============================] - 0s 338us/step - loss: 0.5639 - accuracy: 0.7269
Epoch 3/50
804/804 [==============================] - 0s 306us/step - loss: 0.5608 - accuracy: 0.7287
Epoch 4/50
804/804 [==============================] - 0s 312us/step - loss: 0.5592 - accuracy: 0.7293
Epoch 5/50
733/804 [==========================>...] - ETA: 0s - loss: 0.5581 - accuracy: 0.7300
Epoch 5: saving model to checkpoints_opt2/SPECIAL_CONSIDERATIONS_and_ASK_AMT_dropped/weights.05.hdf5
804/804 [==============================] - 0s 345us/step - loss: 0.5582 - accuracy: 0.7294
Epoch 6/50
804/804 [==============================] - 0s 307us/step - loss: 0.5573 - accuracy: 0.7299
Epoch 7/50
804/804 [==============================] - 0s 322us/step - loss: 0.5567 - accuracy: 0.7294
Epoch 8/50
804/804 [==============================] - 0s 310us/step - loss: 0.5557 - accuracy: 0.7304
Epoch 9/5

,dropped_column,loss,accuracy
8,SPECIAL_CONSIDERATIONS,0.552131,0.739708
14,"SPECIAL_CONSIDERATIONS, ORGANIZATION",0.560411,0.734694
6,STATUS,0.552820,0.734344
7,INCOME_AMT,0.554848,0.732128
15,"SPECIAL_CONSIDERATIONS, STATUS",0.564632,0.731195
13,"SPECIAL_CONSIDERATIONS, USE_CASE",0.560476,0.728630
0,None,0.558676,0.727464
17,"SPECIAL_CONSIDERATIONS, ASK_AMT",0.563169,0.727230
16,"SPECIAL_CONSIDERATIONS, INCOME_AMT",0.559707,0.726764
12,"SPECIAL_CONSIDERATIONS, CLASSIFICATION",0.569426,0.725714


In [17]:
# attempting to drop two columns, using the second best result from single drop as starting point
# Dropping the top result from whole

results_sorted = results_single.sort_values('accuracy', ascending=False)
result_list = list(results_sorted['dropped_column'])
second_drop = result_list[1]

application_df = pre_drop_df.drop(columns=second_drop)

In [18]:

# defining column list to cycle through
columns = application_df.columns[:-1]

for column in columns:

    application_df_new = application_df.drop(columns=column)

    # Generate our categorical variable lists
    cat_var = application_df_new.dtypes[application_df_new.dtypes == 'object'].index.tolist()

    # Encoding DF and merging
    enc = OneHotEncoder(sparse=False)

    encode_df = pd.DataFrame(enc.fit_transform(application_df_new[cat_var]))

    encode_df.columns = enc.get_feature_names_out(cat_var)

    application_df_new = application_df_new.merge(encode_df, left_index=True, right_index=True).drop(columns=cat_var)

    # Splitting data
    y = application_df_new['IS_SUCCESSFUL']
    X = application_df_new.drop(columns=['IS_SUCCESSFUL'])

    X_train, X_test, y_train, y_test = train_test_split(X, y)

    # Scaling Data
    scaler = StandardScaler()

    X_scaler = scaler.fit(X_train)

    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)

    # Define the model
    number_input_features = len(X.columns)
    hidden_nodes_layer1 = 24
    hidden_nodes_layer2 = 12

    nn = tf.keras.models.Sequential()

    # First hidden layer
    nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

    # Second hidden layer
    nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

    # Output layer
    nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

    # Define the checkpoint path and filenames
    os.makedirs(f"checkpoints_opt2/{second_drop}_and_{column}_dropped/",exist_ok=True)
    checkpoint_path = f"checkpoints_opt2/{second_drop}_and_{column}_dropped/weights." + "{epoch:02d}.hdf5"

    # create callback to saves weights every 5 epochs
    cp_callback = ModelCheckpoint(
        filepath=checkpoint_path,
        verbose=1,
        save_weights_only=True,
        save_freq= int(len(X_train)/32*5))
    
    # Train the model
    fit_model = nn.fit(X_train_scaled, y_train, epochs=50,callbacks=[cp_callback])

    model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

    results_df.loc[len(results_df.index)] = [f"{second_drop}, {column}", model_loss, model_accuracy]

results_df.sort_values('accuracy', ascending=False)

Epoch 1/50


/Users/mattzabriskie/miniconda3/envs/mlenv_forge/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


804/804 [==============================] - 1s 319us/step - loss: 0.6086 - accuracy: 0.6857
Epoch 2/50
804/804 [==============================] - 0s 304us/step - loss: 0.5893 - accuracy: 0.7001
Epoch 3/50
804/804 [==============================] - 0s 372us/step - loss: 0.5869 - accuracy: 0.7007
Epoch 4/50
804/804 [==============================] - 0s 303us/step - loss: 0.5851 - accuracy: 0.7018
Epoch 5/50
799/804 [============================>.] - ETA: 0s - loss: 0.5839 - accuracy: 0.7027
Epoch 5: saving model to checkpoints_opt2/STATUS_and_APPLICATION_TYPE_dropped/weights.05.hdf5
804/804 [==============================] - 0s 324us/step - loss: 0.5843 - accuracy: 0.7021
Epoch 6/50
804/804 [==============================] - 0s 304us/step - loss: 0.5838 - accuracy: 0.7017
Epoch 7/50
804/804 [==============================] - 0s 299us/step - loss: 0.5831 - accuracy: 0.7032
Epoch 8/50
804/804 [==============================] - 0s 329us/step - loss: 0.5829 - accuracy: 0.7029
Epoch 9/50
804/8

/Users/mattzabriskie/miniconda3/envs/mlenv_forge/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


804/804 [==============================] - 1s 355us/step - loss: 0.6414 - accuracy: 0.6254
Epoch 2/50
804/804 [==============================] - 0s 367us/step - loss: 0.6214 - accuracy: 0.6479
Epoch 3/50
804/804 [==============================] - 0s 454us/step - loss: 0.6190 - accuracy: 0.6518
Epoch 4/50
804/804 [==============================] - 0s 427us/step - loss: 0.6178 - accuracy: 0.6515
Epoch 5/50
759/804 [===========================>..] - ETA: 0s - loss: 0.6185 - accuracy: 0.6526
Epoch 5: saving model to checkpoints_opt2/STATUS_and_AFFILIATION_dropped/weights.05.hdf5
804/804 [==============================] - 0s 469us/step - loss: 0.6177 - accuracy: 0.6538
Epoch 6/50
804/804 [==============================] - 0s 424us/step - loss: 0.6172 - accuracy: 0.6522
Epoch 7/50
804/804 [==============================] - 0s 405us/step - loss: 0.6169 - accuracy: 0.6541
Epoch 8/50
804/804 [==============================] - 0s 381us/step - loss: 0.6169 - accuracy: 0.6518
Epoch 9/50
804/804 [=

...

/Users/mattzabriskie/miniconda3/envs/mlenv_forge/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


804/804 [==============================] - 1s 317us/step - loss: 0.5915 - accuracy: 0.7017
Epoch 2/50
804/804 [==============================] - 0s 333us/step - loss: 0.5642 - accuracy: 0.7271
Epoch 3/50
804/804 [==============================] - 0s 302us/step - loss: 0.5608 - accuracy: 0.7287
Epoch 4/50
804/804 [==============================] - 0s 303us/step - loss: 0.5590 - accuracy: 0.7300
Epoch 5/50
667/804 [=======================>......] - ETA: 0s - loss: 0.5577 - accuracy: 0.7303
Epoch 5: saving model to checkpoints_opt2/STATUS_and_SPECIAL_CONSIDERATIONS_dropped/weights.05.hdf5
804/804 [==============================] - 0s 307us/step - loss: 0.5581 - accuracy: 0.7307
Epoch 6/50
804/804 [==============================] - 0s 302us/step - loss: 0.5568 - accuracy: 0.7310
Epoch 7/50
804/804 [==============================] - 0s 301us/step - loss: 0.5561 - accuracy: 0.7318
Epoch 8/50
804/804 [==============================] - 0s 303us/step - loss: 0.5557 - accuracy: 0.7322
Epoch 9/50

/Users/mattzabriskie/miniconda3/envs/mlenv_forge/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


804/804 [==============================] - 1s 311us/step - loss: 0.5873 - accuracy: 0.7135
Epoch 2/50
804/804 [==============================] - 0s 298us/step - loss: 0.5672 - accuracy: 0.7259
Epoch 3/50
804/804 [==============================] - 0s 298us/step - loss: 0.5640 - accuracy: 0.7261
Epoch 4/50
804/804 [==============================] - 0s 299us/step - loss: 0.5620 - accuracy: 0.7282
Epoch 5/50
676/804 [========================>.....] - ETA: 0s - loss: 0.5617 - accuracy: 0.7274
Epoch 5: saving model to checkpoints_opt2/STATUS_and_ASK_AMT_dropped/weights.05.hdf5
804/804 [==============================] - 0s 303us/step - loss: 0.5612 - accuracy: 0.7277
Epoch 6/50
804/804 [==============================] - 0s 298us/step - loss: 0.5607 - accuracy: 0.7280
Epoch 7/50
804/804 [==============================] - 0s 299us/step - loss: 0.5598 - accuracy: 0.7285
Epoch 8/50
804/804 [==============================] - 0s 316us/step - loss: 0.5591 - accuracy: 0.7301
Epoch 9/50
804/804 [=====

,dropped_column,loss,accuracy
8,SPECIAL_CONSIDERATIONS,0.552131,0.739708
14,"SPECIAL_CONSIDERATIONS, ORGANIZATION",0.560411,0.734694
6,STATUS,0.552820,0.734344
7,INCOME_AMT,0.554848,0.732128
25,"STATUS, ASK_AMT",0.556427,0.731778
15,"SPECIAL_CONSIDERATIONS, STATUS",0.564632,0.731195
22,"STATUS, ORGANIZATION",0.562458,0.729446
21,"STATUS, USE_CASE",0.556972,0.729096
13,"SPECIAL_CONSIDERATIONS, USE_CASE",0.560476,0.728630
0,None,0.558676,0.727464
